# 台灣農產品交易行情資料下載
批次下載所有蔬菜資料並儲存為 CSV

In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import urllib3

# 停用 SSL 警告
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

vegetable_code_map = {'芥藍': 'LK3'}
code_to_vegetable_name = {code: name for name, code in vegetable_code_map.items()}

def roc_to_western_date(roc_date_str):
    year, month, day = map(int, roc_date_str.split('.'))
    return datetime(year + 1911, month, day)

def western_to_roc_date(date_obj):
    return f"{date_obj.year - 1911:03d}.{date_obj.month:02d}.{date_obj.day:02d}"

def fetch_agri_data_by_chunks(crop_code, start_roc="111.01.01", end_roc="114.07.15", chunk_days=7, delay=0.3):
    start_date = roc_to_western_date(start_roc)
    end_date = roc_to_western_date(end_roc)
    delta = timedelta(days=chunk_days)
    all_data = []
    total_requests = 0
    vegetable_name = code_to_vegetable_name.get(crop_code, crop_code)
    current_date = start_date
    
    while current_date <= end_date:
        query_start = current_date
        query_end = min(current_date + delta - timedelta(days=1), end_date)
        roc_start = western_to_roc_date(query_start)
        roc_end = western_to_roc_date(query_end)
        params = {"Start_time": roc_start, "End_time": roc_end, "CropCode": crop_code}
        
        try:
            response = requests.get(
                "https://data.moa.gov.tw/api/v1/AgriProductsTransType/", 
                params=params, 
                timeout=30,
                verify=False  # 停用 SSL 驗證
            )
            response.raise_for_status()
            data = response.json().get("Data", [])
            total_requests += 1
            if data:
                all_data.extend(data)
                print(f"✓ {roc_start} ~ {roc_end}: 取得 {len(data)} 筆")
        except Exception as e:
            print(f"✗ 錯誤於 {roc_start} ~ {roc_end}: {e}")
        
        current_date += delta
        if current_date <= end_date:
            time.sleep(delay)
    
    if all_data:
        df = pd.DataFrame(all_data)
        if 'TransDate' in df.columns:
            df = df.drop_duplicates(subset=['TransDate', 'CropCode', 'MarketName'], keep='first')
            try:
                df['TransDate_parsed'] = pd.to_datetime(df['TransDate'], format='%Y/%m/%d', errors='coerce')
                df = df.sort_values('TransDate_parsed', ascending=True)
                df = df.drop('TransDate_parsed', axis=1)
            except:
                pass
        return df
    return pd.DataFrame()

def fetch_all_vegetables_and_save(start_roc="114.01.01", end_roc="114.07.15", chunk_days=5, delay=0.3):
    for veg_name, crop_code in vegetable_code_map.items():
        print(f"🌿 正在下載：{veg_name}（{crop_code}）")
        df = fetch_agri_data_by_chunks(crop_code, start_roc, end_roc, chunk_days, delay)
        if not df.empty:
            filename = f"{veg_name}.csv"
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"✅ 已儲存：{filename}，共 {len(df)} 筆資料")
        else:
            print(f"⚠️ 無資料：{veg_name}")

fetch_all_vegetables_and_save()
fetch_all_vegetables_and_save()

🌿 正在下載：芥藍（LK3）
✓ 114.01.01 ~ 114.01.05: 取得 51 筆
✓ 114.01.06 ~ 114.01.10: 取得 42 筆
✓ 114.01.11 ~ 114.01.15: 取得 44 筆
✓ 114.01.16 ~ 114.01.20: 取得 35 筆
✓ 114.01.21 ~ 114.01.25: 取得 55 筆
✓ 114.01.26 ~ 114.01.30: 取得 30 筆
✓ 114.01.31 ~ 114.02.04: 取得 21 筆
✓ 114.02.05 ~ 114.02.09: 取得 39 筆
✓ 114.02.10 ~ 114.02.14: 取得 23 筆
✓ 114.02.15 ~ 114.02.19: 取得 40 筆
✓ 114.02.20 ~ 114.02.24: 取得 36 筆
✓ 114.02.25 ~ 114.03.01: 取得 49 筆
✓ 114.03.02 ~ 114.03.06: 取得 35 筆
✓ 114.03.07 ~ 114.03.11: 取得 42 筆
✓ 114.03.12 ~ 114.03.16: 取得 46 筆
✓ 114.03.17 ~ 114.03.21: 取得 37 筆
✓ 114.03.22 ~ 114.03.26: 取得 36 筆
✓ 114.03.27 ~ 114.03.31: 取得 41 筆
✓ 114.04.01 ~ 114.04.05: 取得 30 筆
✓ 114.04.06 ~ 114.04.10: 取得 30 筆
✓ 114.04.11 ~ 114.04.15: 取得 37 筆
✓ 114.04.16 ~ 114.04.20: 取得 48 筆
✓ 114.04.21 ~ 114.04.25: 取得 39 筆
✓ 114.04.26 ~ 114.04.30: 取得 40 筆
✓ 114.05.01 ~ 114.05.05: 取得 38 筆
✓ 114.05.06 ~ 114.05.10: 取得 45 筆
✓ 114.05.11 ~ 114.05.15: 取得 41 筆
✓ 114.05.16 ~ 114.05.20: 取得 41 筆
✓ 114.05.21 ~ 114.05.25: 取得 50 筆
✓ 114.05.26 ~ 114.05.30: 取得 